In this notebook will analyze plate reader data assessing leaky T7 RNAP expression of variations of the differentiation design and generate the plots used to make Supplementary Figure 3.

In [1]:
import numpy as np
import pandas as pd
import scipy.optimize
import murraylab_tools.biotek as btek
import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from cycler import cycler
import numpy as np

sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 1.5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})


mpl.rc('axes', prop_cycle=(cycler('color', ['r', 'k', 'b','g','y','m','c']) ))

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

tw = 1.5
sns.set_style({"xtick.major.size": 3, "ytick.major.size": 3,
               "xtick.minor.size": 2, "ytick.minor.size": 2,
               'axes.labelsize': 16, 'axes.titlesize': 16,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', linewidth=1.5)
mpl.rc('legend', fontsize=14)
mpl.rc('legend', frameon=False)
mpl.rc('figure', figsize=(8.5,15))
%matplotlib qt

In [2]:
# tidy data, tidy df can be read as csv below
btek.tidy_biotek_data('./20210814_diff_leak_experiment.csv',convert_to_uM=False,volume=300,supplementary_filename='20210814_metadata.csv')

/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well G1; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well G2; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well G3; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well G4; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well G5; throwing out data for that well.
  % well_name)
/Users/rorywilliams/

In [3]:
data = pd.read_csv('20210814_diff_leak_experiment_tidy.csv')
OD = data.loc[data.Channel=='OD600',:]
GFP = data.loc[(data.Channel=='sfGFP')&(data.Gain==61),:]

In [4]:
# back subtract data and get OD normalized fluoresence
OD_endpoint = OD.loc[OD['Time (hr)']==OD['Time (hr)'].max(),:]
OD_bg = 0.095
OD_endpoint['Measurement'] -= OD_bg
GFP_endpoint = GFP.loc[GFP['Time (hr)']==GFP['Time (hr)'].max(),:]
GFP_bg = np.array([99,110,95]).mean()
GFP_endpoint['Measurement'] -= GFP_bg
GFP_endpoint['GFP/OD'] = GFP_endpoint['Measurement'].values/OD_endpoint['Measurement'].values

/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

Make plot as shown in Supplementary Figure 3

In [5]:
sns.set_palette(['black','#d73027','#fc8d59','#91bfdb','#4575b4'])
strains = ['diff FL T7RNAP 1','diff T13m FL T7RNAP 2','diff split T7RNAP','JS006']
cole1s = ['pT7 GFP','pT7 GFP/pSal Bxb1']
fig, ax = plt.subplots(1,4,figsize=(15,3),sharex=True,sharey=True)
for i, strain in enumerate(strains):
    ax[i].set_title(strain,fontsize=14)
    data = GFP_endpoint.loc[GFP_endpoint.strain==strain,:]
    sns.swarmplot(ax=ax[i],x="cole1", y="GFP/OD", hue="sal", data=data,dodge=True,size=6,alpha=0.8)
    ax[i].get_legend().set_visible(False)
    ax[i].set_ylabel('')
    ax[i].set_ylabel('')
ax[0].set_yscale('log')
ax[0].legend(['-sal','100$\mu$M sal'],fontsize=12)
ax[0].set_ylabel('GFP/OD (a.u.)',fontsize=14)
ax[0].tick_params(axis='y',labelsize=12)
for i in range(4):
    ax[i].tick_params(axis='x',labelsize=12)
plt.savefig('./20210814_diff_variants_leak.pdf')   #uncomment to save figure
